In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
from tqdm import tqdm

from sae_lens import SAE
# from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
# from sae_lens import SparseAutoencoder, ActivationsStore

# from steering.eval_utils import evaluate_completions
from steering.utils import normalise_decoder
from steering.patch import generate

# from sae_vis.data_config_classes import SaeVisConfig
# from sae_vis.data_storing_fns import SaeVisData

# import plotly.express as px

torch.set_grad_enabled(False)

/Users/slava/safety/steering_research/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import List, Callable, Union, Optional, Literal

from functools import partial
from tqdm import tqdm
import json

import torch
from torch.utils.data import DataLoader
import einops

from datasets import load_dataset

from transformer_lens import HookedTransformer
from transformer_lens.utilities import devices
from transformer_lens.past_key_value_caching import HookedTransformerKeyValueCache
import transformer_lens.utils as tutils

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained("gemma-2b", device=device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]


Loaded pretrained model gemma-2b into HookedTransformer


In [4]:
hp6 = "blocks.6.hook_resid_post"
hp12 = "blocks.12.hook_resid_post"

sae6, _, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb", # see other options in sae_lens/pretrained_saes.yaml
    sae_id = hp6, # won't always be a hook point
    device = 'cpu'
)

sae6 = sae6.to(device)
normalise_decoder(sae6)

In [5]:
intelligence = sae6.W_dec[10351]   # intelligence and genius
writing = sae6.W_dec[1058]  # writing
anger = sae6.W_dec[1062]  # anger
london = sae6.W_dec[10138]  # London
wedding = sae6.W_dec[8406]  # wedding


In [12]:
def patch_resid(resid, hook, steering, scale=1, some_text=""):
    print(some_text)
    resid[:, :, :] = resid[:, :, :] + steering * scale
    return resid

hooks = [(
    "blocks.6.hook_resid_post", 
    partial(patch_resid, steering=london, scale=70, some_text="London")
), 
(
    "blocks.6.hook_resid_post", 
    partial(patch_resid, steering=wedding, scale=70, some_text="wedding")
)
]

In [13]:
generate(model,
        hooks=hooks,
        schedules=[(1, 15), (16, 30)],
        max_length=35,
        prompt="I think",
         )

London
London
London
London
London
London
London
London
London
London
London
London
London
London
London
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding
wedding


['I think there is a bug, if one of the points/stations was in the London area.\nMaybe the date is a big problem between to guests.\n\nHello,\n\nyou',
 'I think the London is better, I went to the theatre with two friends, as a birthday present to our friend. It was fun but not at all the best night we have ever',
 "I think it would be very easy to do a London to Paris marathon, if I wasn't a runner but wedding planner or photographer. As a Londoner, London was on my",
 'I think it could be done in 10 mins. So in 3 years you’ll pay almost $700 for the same service (assuming its still the same).']